In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# <div style="text-align: center; background-color: #595964; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">📊EDA |Life Expectancy  | Visualization </div>

# <div style="text-align: center; background-color: #00BFFF; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">1. Import Necessary Libraries</div>

In [ ]:
!pip install altair
!pip install folium
!pip install pycountry-convert
!pip install geopy

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import missingno as mno
import plotly.offline as pyo 
import plotly.figure_factory as ff
import plotly.io as pio
from wordcloud import WordCloud
color_pal = sns.color_palette()
plt.style.use('seaborn-dark-palette')
plt.style.use('dark_background')

import nltk
import re

import warnings
warnings.filterwarnings('ignore')
sns.set_theme(style='darkgrid', palette='colorblind')
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()

#Model
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
df=pd.read_csv('/kaggle/input/life-expectancy-at-birth-across-the-globe/Life Expectancy at Birth.csv')
df=pd.read_csv('/kaggle/input/life-expectancy-at-birth-across-the-globe/Life Expectancy at Birth.csv')

<a id="1"></a>
# <div style="text-align: center; background-color: #00BFFF; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">2. Exploratory Data Analysis 📊</div>

In [ ]:
df.head()

In [ ]:
col_names = df.columns.to_list()
df.rename(columns={ x:str(re.search( r'\((\d+)\)',x).group(1)) for x in col_names[7:]} , inplace=True)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(include='object')

In [ ]:
unique_values = df.nunique()
typpe_values = df.dtypes

summary_df = pd.DataFrame({'Unique Values': unique_values, 'Data Type': typpe_values})
print(summary_df)

In [ ]:
df.shape

# <div style="text-align: center; background-color: #6495ED; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">3. Null values</div>


In [ ]:
df.isna().sum()

# <div style="text-align: center; background-color: #6495ED; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">4. Duplicate rows</div>

In [ ]:
# Finding duplicate rows
duplicate_rows = df[df.duplicated(keep='first')]

# Number of duplicate rows
num_duplicates = duplicate_rows.shape[0]

# Displaying the duplicate rows
print(f"Number of duplicate rows: {num_duplicates}")
duplicate_rows

In [ ]:
grouped_df = df.groupby("Continent")["HDI Rank (2021)"].mean().reset_index()
grouped_df.head()

# <div style="text-align: center; background-color: #6495ED; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">5. Data visualisation</div>


In [ ]:
# Import the ProfileReport class from the ydata_profiling library
from ydata_profiling import ProfileReport

# Create a comprehensive profile report for the DataFrame 'df'
# This report will contain various statistics, insights, and visualizations about the data
profile = ProfileReport(df)


In [ ]:
profile                     # or this one
profile.to_notebook_iframe() # use this line to show the output

In [ ]:
import pandas as pd
import plotly.express as px

# Assuming you have a DataFrame named 'df'

# Columns for which you want to create plots
columns_to_plot = ['Continent', 'UNDP Developing Regions', 'Hemisphere']

for column in columns_to_plot:
    top10_values = df[column].value_counts()[:10]
    
    # Create a bar plot using Plotly Express
    fig = px.bar(
        x=top10_values.index,  # Use index as x values
        y=top10_values.values,
        color=top10_values.index,  # Use index for color variation
        labels={'x': column, 'y': 'Count'},
        title=f'Top 10 {column} Distribution',
        template='plotly_white'
    )
    fig.update_layout(
        font=dict(size=17, family="Franklin Gothic")
    )
    
    # Display the plot
    fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

# Assuming you have a DataFrame named 'df'

# Columns and y-labels for which you want to create plots
columns_to_plot = [
    ('Country', 'HDI Rank (2021)', 'HDI Rank (2021) by Country'),
    ('Continent', 'HDI Rank (2021)', 'HDI Rank (2021) by Continent'),
    ('Human Development Groups', 'HDI Rank (2021)', 'HDI Rank (2021) by Human Development Groups')
]

for column, y_label, title in columns_to_plot:
    data = df.groupby(column)[y_label].sum().reset_index()
    
    # Create a line plot using Plotly Express
    fig = px.line(
        data,
        x=column,
        y=y_label,
        title=title,
        height=800,
        line_shape='linear',  # Change the line shape (linear, spline, hv, vh, etc.)
        color_discrete_sequence=px.colors.qualitative.Pastel1  # Change the color palette
    )
    fig.update_layout(
        xaxis_title=column,
        yaxis_title=y_label,
        font=dict(size=17, family="Franklin Gothic"),
        template='plotly_white'  # Change the background template (plotly, ggplot2, seaborn, etc.)
    )
    
    # Display the plot
    fig.show()


In [ ]:
# x-axis: "work_year", y-axis: "salary_in_usd", color-coded by "experience_level",
# and create a notched box plot
fig = px.box(df, x="Continent", y="HDI Rank (2021)", color="Human Development Groups", notched=True)

# Update the layout of the plot
fig.update_layout(
    title_text='HDI Rank (2021) Box Distribution',  # Set the title of the plot
    height=500,  # Set the height of the plot
    xaxis_title="Continent",  # Label for the x-axis
    yaxis_title="HDI Rank (2021)",  # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic")  # Set the font size and family for the text
)

# Display the plot
fig.show()

In [ ]:
# Columns and titles for which you want to create sunburst charts
columns_to_plot = [
    ('Country', 'Distribution of Country'),
    ('Continent', 'Distribution of Continent'),
    ('Hemisphere','Distribution of Human Hemisphere')
]

for column, title in columns_to_plot:
    # Create a sunburst chart using Plotly Express
    fig = px.sunburst(df, path=[column], color_discrete_sequence=px.colors.qualitative.Set1
                     )
    fig.update_layout(title_text=title, height=500)
    
    # Display the plot
    fig.show()

In [ ]:
import pandas as pd
import plotly.express as px

# Assuming you have a DataFrame named 'df'

# Drop rows with missing values in the specified columns
df_cleaned = df.dropna(subset=["HDI Rank (2021)", "Human Development Groups"])

# Create a scatter plot using Plotly Express
fig = px.scatter(
    df_cleaned,
    x="HDI Rank (2021)",
    y="Human Development Groups",
    size="HDI Rank (2021)",
    color="Continent",
    hover_name="Country",
    log_x=True,
    size_max=60
)
fig.show()


In [ ]:
df.head()

In [ ]:
# Melt the dataframe to make it suitable for animation
melted_df = df.melt(id_vars=['ISO3', 'Country', 'Continent'],
                    value_vars=[str(year) for year in range(1990, 2022)],
                    var_name='Year', value_name='Life Expectancy')

# Create an animated map visualization
fig = px.choropleth(
    melted_df,
    locations="ISO3",
    color="Life Expectancy",
    hover_name="Country",
    hover_data=["Continent", "Year"],
    animation_frame="Year",
    title="Life Expectancy at Birth by Continent (1990-2021)",
    color_continuous_scale="viridis",
    range_color=(40, 90),  # Adjust the color scale range
    projection="natural earth"
)

# Update layout
fig.update_layout(geo=dict(showcoastlines=True))

# Show the animated map
fig.show()

In [ ]:
import altair as alt
import re
from altair import datum
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2
from geopy.geocoders import Nominatim

In [ ]:
import plotly.express as px
fig1 = px.choropleth(df,
                     locations='Country',
                     locationmode='country names',
                     color='2021',
                     color_continuous_scale=px.colors.sequential.Viridis,
                     template='plotly_dark',
                     title = '2021 Life Expectancy At Birth')

fig1.update_layout(font = dict(size = 17, family="Franklin Gothic"))

fig1.show()

In [ ]:
fig2 = px.choropleth(df,
                     locations='Country',
                     locationmode='country names',
                     color='Human Development Groups',
                     template='plotly_white',
                     title = 'Human Development Groups')

fig2.update_layout(font = dict(size = 17, family="Franklin Gothic"))

fig2.show()


# <div style="text-align: center; background-color: #6495ED; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">6. Categorical</div>

In [ ]:
# Select columns with object (categorical) data types
num_cols = df.select_dtypes(include='object').columns.tolist()

# Initialize the LabelEncoder
le = LabelEncoder()

# Apply Label Encoding to the selected numerical columns
for x in num_cols:  
    df[x] = le.fit_transform(df[x])

# Now, your categorical columns (excluding column 0) have been converted to numerical values
df.head()


# <div style="text-align: center; background-color: #6495ED; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">7.HeatMap & Most Positively & Most Negatively</div>


In [ ]:
Corr_Matrix = df.corr()

# Mask the lower triangular part of the matrix
#mask = np.triu(np.ones(Corr_Matrix.shape), k=1)

# Set up the figure and plot the heatmap
plt.figure(figsize=(60, 6))  # Adjust the figure size as needed
sns.heatmap(Corr_Matrix.head(), annot=True, cmap='BuPu')
plt.title('Correlation Heatmap (Head)')
plt.show()

# Most Positively

In [ ]:
print('Top 5 Most Positively Correlated to the To Continent')
Corr_Matrix['Continent'].sort_values(ascending=False).head(5)

# Most Negatively

In [ ]:
print('Top 5 Most Positively Correlated to Continent ')
Corr_Matrix['Continent'].sort_values(ascending=True).head(5)

In [ ]:
df=df.dropna()
df.shape

# <div style="text-align: center; background-color: #6495ED; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">9. spliting the dataset

</div>

In [ ]:
X = df.drop(columns=['Continent'])
y = df['Continent']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Display the shapes of the resulting datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


# <div style="text-align: center; background-color: #6495ED; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">10. Model Building and Analysis

</div>

In [ ]:
models = {
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
}
best_model = None
best_r2 = 0

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred= model.predict(X_test)

    # Evaluate the model
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    submit = pd.DataFrame()
    submit['Actual_salary'] = y_test
    submit['Predict_salary'] = y_pred
    submit = submit.reset_index()
    r2 = r2_score(y_test, y_pred)
    if r2 > best_r2:
        best_r2 = r2
        best_model = model.__class__.__name__

    print(f'{model_name}:')
    print(f'R2 Score: {r2:.2f}')
    print(f'Mean Absolute Error (MAE): {mae:.2f}')
    print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')
    print(submit.head(5))

    print('----------------------------------------')
print(f"The best performing model is: {best_model} with accuracy: {best_r2:.2f}")


# <div style="text-align: center; background-color: #6495ED; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">11. feature importances

</div>


In [ ]:
import matplotlib.pyplot as plt

importances = model.feature_importances_
feature_names = X.columns
feature_importance_dict = dict(zip(feature_names, importances))
sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

top_n = 5  # Set the number of top features to display
top_feature_names, top_importances = zip(*sorted_feature_importance[:top_n])

fig = px.bar(
    x=top_importances,
    y=top_feature_names,
    orientation='h',
    title='Top 5 Feature Importance',
    labels={'x': 'Importance', 'y': 'Feature'},
    color=top_importances,  # Color bars by importance values
    color_continuous_scale='reds',  # Choose a color scale
)

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')

fig.show()